In [ ]:
!git clone https://github.com/siddhant230/DiffTimesNet.git

In [ ]:
%cd DiffTimesNet/
!git checkout dev-model-data

In [ ]:
!git branch

### start

# get all datasets and save

In [ ]:
!python get_data.py

# dependencies

In [ ]:
!pip install -q -r requirements.txt

## TASKS

### classification

In [ ]:
!chmod +x ./scripts/classification/TimesNet.sh
!./scripts/classification/TimesNet.sh

###long term forecast

In [ ]:
!chmod +x ./scripts/long_term_forecast/ETT_script/TimesNet_ETTh1.sh
!./scripts/long_term_forecast/ETT_script/TimesNet_ETTh1.sh

###short-term forecast

In [ ]:
!chmod +x ./scripts/short_term_forecast/TimesNet_M4.sh
!./scripts/short_term_forecast/TimesNet_M4.sh

###imputation

In [ ]:
!chmod +x ./scripts/imputation/ETT_script/TimesNet_ETTh1.sh
!./scripts/imputation/ETT_script/TimesNet_ETTh1.sh

###anomaly detection

In [ ]:
!chmod +x ./scripts/anomaly_detection/PSM/TimesNet.sh
!./scripts/anomaly_detection/PSM/TimesNet.sh